In [ ]:
import openai
from dotenv import load_dotenv
from github import Github, GithubException
import re

In [ ]:
def parse_python_code(input_str):
    pattern = r'```python\n(.*?)\n```'
    result = re.search(pattern, input_str, re.DOTALL)
    if result:
        return result.group(1)
    else:
        return "No match found"

In [ ]:
load_dotenv()

In [ ]:
# Login and get repo
repo_path = "Scott-Huston/gpt_dev"

g = Github(os.getenv("GH_TOKEN"))
repo = g.get_repo(repo_path)

In [ ]:
# Get file paths for all files in repo
# get_contents only gets root level at first

filepaths = []
contents = repo.get_contents("")
while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
        # Add contents of subfolder
        contents.extend(repo.get_contents(file_content.path))
    else:
        if file_content.path != ".gitignore": # gitignore info just unnecessary tokens
            filepaths.append(file_content.path)

filepaths

In [ ]:
# Represent contents of repo in a long string
repo_content_string = ""

for filepath in filepaths:
    repo_content_string += f"FILE PATH:\n{filepath}\nFILE CONTENT:\n"

    file_content = repo.get_contents(filepath).decoded_content
    repo_content_string += f"{file_content}\n\n"

repo_content_string

In [ ]:
# Authenticate
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
system_message = "You are a world-class 10X programmer." 


task_description = "Please take the general concept of the code in the test.ipynb notebook and turn it into well structured code with a script that can be executed in a user-friendly manner from the command line. Be sure to update the readme with good documentation and instructions as well"

clarification_prompt = "Do not attempt to complete the task yet. Are there any questions you have about the task before you begin?"

first_message = f"""Here are the contents of a code repository on Github:
{repo_content_string}
Task Description:
{task_description}
{clarification_prompt}
"""

response = openai.ChatCompletion.create(
  model="gpt-4-32k",
  messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": first_message}
    ]
)

In [ ]:
len(first_message.split(" "))

In [ ]:
source_branch = '{repo.default_branch}'
target_branch = 'test'

try:
    # Test to see if branch already exists
    repo.get_branch("test")
except GithubException as e:
    if e.status == 404:
        # If branch does not exist, create it
        sb = repo.get_branch(source_branch)
        repo.create_git_ref(ref='refs/heads/' + target_branch, sha=sb.commit.sha)

repo.get_branch("test")

In [ ]:
create_pr_message = f"""Ok, now please create a pull request to complete the task using the PyGithub Python library. You should create a new branch, create/modify/delete files as necessary, and then create a pull request.

# Example of creating a branch

source_branch = '{repo.default_branch}'
target_branch = 'test'

try:
    # Test to see if branch already exists
    repo.get_branch("test")
except GithubException as e:
    if e.status == 404:
        # If branch does not exist, create it
        sb = repo.get_branch(source_branch)
        repo.create_git_ref(ref='refs/heads/' + target_branch, sha=sb.commit.sha)

# Example of creating a new file
file_content = "This is the file content"
commit_message = "create test file"
repo.create_file(path="test.txt", message=commit_message, content=file_content, branch="test")

# Example of updating a file
contents = repo.get_contents("test.txt", ref="test")
repo.update_file(contents.path, "more tests", "more tests", contents.sha, branch="test")

# Example of deleting a file
contents = repo.get_contents("test.txt", ref="test")
repo.delete_file(contents.path, message="remove test", sha=contents.sha, branch="test")

# Example of creating the PR
body = '''
This PR created, updated and deleted a test file
'''
repo.create_pull(title="Test file", body=body, head="test", base=repo.default_branch)


YOUR ENTIRE RESPONSE SHOULD BE EXCLUSIVELY PYTHON CODE. YOUR RESPONSE MUST RUN IN PYTHON AS-IS TO CREATE THE PULL REQUEST.

Your response will be run in a python script after the following code has already been run.

from github import Github, GithubException

# Login and get repo
g = Github(os.getenv("GH_TOKEN"))
repo = g.get_repo({repo_path})

"""

response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": first_message},
        {"role": "assistant", "content": first_response_content},
        {"role": "user", "content": create_pr_message}
    ]
)

In [ ]:
## Need to separate out the middle of ```python\n XXXX \n```
## Probably need to specify to not import any non-standard packages
## Specifying that the response needs to be in python might be confusing if the repo is in another language
## Wrap calls to gpt-4 in try/except to handle connection issues

In [ ]:
exec(parse_python_code(create_pr_response))